In [33]:
from cn_clip.clip.configuration_bert import BertConfig
config = BertConfig.from_json_file('config.json')

In [34]:
from cn_clip.clip.modeling_bert import BertModel
model = BertModel(config)

In [35]:
import torch
static_model = torch.load('new_pytorch_model.bin')

In [16]:
# 去除ernie的前缀，并保存为新的模型
new_model = {}
for k, v in static_model.items():
    if k == 'ernie.embeddings.task_type_embeddings.weight':
        continue
    if k.startswith('ernie'):
        # 替换gamma为weight， beta为bias
        if k.endswith('gamma'):
            k = k.replace('gamma', 'weight')
        elif k.endswith('beta'):
            k = k.replace('beta', 'bias')
        # 删除前缀ernie
        k = k.replace('ernie.', '')
        new_model[k] = v

In [36]:
model.load_state_dict(static_model)

<All keys matched successfully>

In [18]:
torch.save(model.state_dict(), 'new_pytorch_model.bin')

In [13]:
new_model_state_dict = torch.load('new_pytorch_model.bin')

In [45]:
from transformers import ErnieModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("nghuyong/ernie-3.0-mini-zh")
ernie = ErnieModel.from_pretrained("nghuyong/ernie-3.0-mini-zh")

In [46]:
from cn_clip.clip.bert_tokenizer import FullTokenizer
new_tokenizer = FullTokenizer(vocab_file='vocab.txt')

In [52]:
inputs = tokenizer("我爱北京天安门", return_tensors="pt")
new_inputs = [new_tokenizer.vocab['[CLS]']] + new_tokenizer.convert_tokens_to_ids(new_tokenizer.tokenize("我爱北京天安门"))  + [new_tokenizer.vocab['[SEP]']]
outputs = ernie(**inputs)
new_outputs = model(torch.tensor([new_inputs]))

In [53]:
isEquals = torch.equal(outputs['last_hidden_state'], outputs[0])

In [41]:
import os
os.path.join(os.path.dirname(os.path.abspath(__file__)), "vocab.txt")

NameError: name '__file__' is not defined